<a href="https://colab.research.google.com/github/honeypatel1704/student-management/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask
!pip install pymongo
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.8 MB/s eta 0:00:00


In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from pymongo import MongoClient
from datetime import datetime

In [ ]:
app = Flask(__name__)

In [ ]:
# MongoDB configuration
client = MongoClient('mongodb://localhost:27017/')
db = client['student_db']
students = db['students']

In [ ]:
@app.route('/')
def home():
    return render_template('base.html')

In [ ]:
@app.route('/add_student', methods=['GET', 'POST'])
def add_student():
    if request.method == 'POST':
        student_data = {
            'name': request.form['name'],
            'father_name': request.form['father_name'],
            'mother_name': request.form['mother_name'],
            'birthdate': datetime.strptime(request.form['birthdate'], '%Y-%m-%d'),
            'address': request.form['address'],
            'student_phone': request.form['student_phone'],
            'father_phone': request.form['father_phone'],
            'enrollment_number': request.form['enrollment_number'],
            'class': request.form['class'],
            'batch': request.form['batch'],
            'email': request.form['email'],
            'linkedin': request.form.get('linkedin', ''),
            'github': request.form.get('github', ''),
            'created_at': datetime.now()
        }
        students.insert_one(student_data)
        return redirect(url_for('home'))
    return render_template('add_student.html')

In [ ]:
@app.route('/search', methods=['GET', 'POST'])
def search():
    if request.method == 'POST':
        query = {}
        if request.form['search_term']:
            query['$or'] = [
                {'name': {'$regex': request.form['search_term'], '$options': 'i'}},
                {'enrollment_number': request.form['search_term']}
            ]
        results = list(students.find(query))
        return render_template('search.html', results=results)
    return render_template('search.html', results=None)

In [ ]:
@app.route('/student/<enrollment_number>')
def student_details(enrollment_number):
    student = students.find_one({'enrollment_number': enrollment_number})
    if student:
        # Convert date to string for display
        student['birthdate'] = student['birthdate'].strftime('%Y-%m-%d')
        return render_template('student_details.html', student=student)
    return "Student not found", 404

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
